In [ ]:
#Deployment & Decision-making 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# -----------------------------
# 1. User selects ambient pressure
# -----------------------------
user_input = input("Enter ambient pressure to evaluate (0.01 for Vacuum, 1.01325 for Ground): ")
try:
    ambient_pressure = float(user_input)
except:
    raise ValueError("Please enter a valid numeric ambient pressure (0.01 or 1.01325).")

# -----------------------------
# 2. Load corresponding dataset
# -----------------------------
if ambient_pressure == 0.01:
    df = pd.read_csv("vacuum_statistics.csv")
    print("Loaded Vacuum data.")
elif ambient_pressure == 1.01325:
    df = pd.read_csv("ground_statistics.csv")
    print("Loaded Ground data.")
else:
    raise ValueError("Invalid ambient pressure. Must be 0.01 (Vacuum) or 1.01325 (Ground).")

# -----------------------------
# 3. Clean column names
# -----------------------------
df.columns = df.columns.str.strip()

# -----------------------------
# 4. Optional: create derived features
# -----------------------------
# You mentioned combining heat capacity, temperature, and oxidizer/fuel ratio
df['heat_temp_of_ratio'] = df['heat_capacity_ratio'] * df['combustion_temperature_K'] * df['oxidizer_fuel_ratio']

# Include mass oxidizer flow if it exists
if 'oxidizer_mass_flow_kg_s' not in df.columns:
    df['oxidizer_mass_flow_kg_s'] = 0  # placeholder if missing

# -----------------------------
# 5. Define features and target
# -----------------------------
features = [
    "chamber_pressure_bar",
    "oxidizer_fuel_ratio",
    "combustion_temperature_K",
    "heat_capacity_ratio",
    "specific_impulse_s",
    "oxidizer_mass_flow_kg_s",
    "heat_temp_of_ratio"
]

# Drop rows with missing values in features or target
df = df.dropna(subset=features + ['combustion_stability_margin'])

X = df[features]
y = df['combustion_stability_margin']

# -----------------------------
# 6. Split dataset
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------
# 7. Train linear regression model
# -----------------------------
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# -----------------------------
# 8. Make predictions
# -----------------------------
y_pred = lr_model.predict(X_test)

# -----------------------------
# 9. Evaluate model
# -----------------------------
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\nLinear Regression Model Evaluation")
print("---------------------------------")
print(f"Ambient Pressure: {ambient_pressure} bar")
print(f"RMSE: {rmse:.4f}")
print(f"R^2 Score: {r2:.4f}")

# -----------------------------
# 10. Show coefficients
# -----------------------------
coefficients = pd.DataFrame({
    "Feature": features,
    "Coefficient": lr_model.coef_
})
print("\nModel Coefficients:")
print(coefficients)